## SARSA Algorithm to recommend songs without state and action approximation

In this notebook, we generated the used the autoencoder for state and action space to generate the latent vector representation of both of these entities. Then we used the latent representaion itself as state and action space for learing.

#### Importing necessary libraries for this notebook

In [9]:
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Input, Dense
from keras.models import Model
import matplotlib.pyplot as plt
import random
import math
from numpy import zeros, newaxis
from statistics import mean 
%matplotlib inline

Extracting the latent representation of each track stored in latent_tack_features.csv

In [2]:
latent_track_features_df = pd.read_csv("latent_track_features.csv");
latent_track_features_df.head()

,track_id,0,1,2,3,4,5,6,7
0,t_f9b9a072-8dae-4816-bfd6-d91fc75a5744,0.085536,0.023943,0.125375,-0.145933,0.449010,-0.106019,-0.190885,0.282416
1,t_7f435320-5d3b-497d-9346-0c97429af2b9,0.152102,-0.118725,-0.168167,0.291125,0.540807,0.042686,-0.288468,0.130090
2,t_949441c4-4920-4eed-b929-2d67d37b7bd9,0.056647,-0.077121,0.052887,-0.076303,0.644329,-0.148826,-0.292301,0.208705
3,t_63eae5ac-744a-46ee-b719-0db8e70d6f7f,0.120041,-0.071038,-0.012514,-0.061835,0.621581,-0.082328,-0.210929,0.185848
4,t_3c2c29b5-c738-42c0-83e1-ee0e88ac834d,0.075071,0.094992,0.372702,0.001345,0.534761,-0.017185,-0.420036,0.035725


Creating a dictionary of track_id and latent vector for easy search

In [3]:
latent_track_features_dict = latent_track_features_df.set_index('track_id').T.to_dict('list')

Reading session data from session_track_reward_data.csv created during sampling process.

In [4]:
session_df = pd.read_csv("session_track_reward_data.csv");
session_df.head()

,session_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,53_0068dc30-11ee-4417-b045-396c627ef013,t_6131a810-b1c3-4be9-92c9-54105e775151,1,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,1,t_34871798-7470-4bf7-85a4-e0c8e277a2d3,1,t_2c06c4b3-f2a4-4ccc-b673-b9943aa7e1e3,0,t_94e173bf-9244-4c4b-b9a8-2a735a3d6ccf,...,t_7c13b9d7-39b7-48e0-828f-6c920ac6535f,1,t_17ee7a20-975e-4785-8e31-10d0da37457b,0,t_6e6b433c-59da-49bc-9bdf-ddc122928277,1,t_6e6b433c-59da-49bc-9bdf-ddc122928277,0,t_309812b2-11d2-47ed-90fb-c0dbf25b0866,0
1,53_006d0a57-11b2-42dd-98b6-625bb28283f6,t_8c1a8a18-7181-4364-bca4-94b97e5dc2d8,0,t_665cc6c4-7745-47bc-bb65-11eeb33e1ecc,0,t_a646aa9c-0c00-45a5-aa53-cb428e171251,1,t_69ef057b-3e0c-469e-a71a-4e348f6b6aed,0,t_2d9aef46-0e33-479f-ace4-019793ad5ea5,...,t_33831664-66b6-4590-9939-052d3c8c4b11,0,t_13ceba8d-4d30-44c7-a80d-9f75adca2432,0,t_6966e4af-fbdc-4e95-ab49-6af31c95f37a,0,t_26eff975-cb7e-4507-a3b6-278b526584bc,0,t_68613f0d-1617-4bd6-aacb-372a79ddd770,1
2,53_038a6aaa-7d85-4bab-bd25-2f69a3eb12b6,t_6838642c-b80e-4b0b-b18f-5389fb74d00f,0,t_d1f175e8-a157-411c-9447-d48615e71ed0,0,t_da1fef49-47b5-4236-842a-36347cb78460,0,t_e000b439-a7e6-4e85-84b7-4e3d54742d60,0,t_d55fd565-fe40-4442-b8f0-393e9222d23e,...,t_88e8671d-e6ee-48dc-b610-6d1cad2657cf,1,t_b7954b03-6717-42f4-a00d-fcaa26ac7f90,1,t_0337a921-6153-4c8b-8c27-679a3796dfc8,1,t_93cccc0a-8dcb-4697-b501-dc9056309abc,1,t_bf81cf32-8335-457d-a867-5fbaca8d56b9,1
3,53_040eeedd-a7ce-4229-a1c0-17e510ea9f76,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_4c316d3c-ab2e-4943-b0cd-c59a0c7480b4,0,t_d9b73bdb-eecf-4c8c-9f85-c2bd3bab8d33,0,t_59647f23-dd08-45d1-8eeb-03fdd9c1b718,0,t_ad62139c-561c-47d9-9481-744b570d74e7,...,t_d1f58de5-01c0-4bda-962c-62fed400e2f4,0,t_18b618d9-ad3e-4d24-8e6e-8e3c2c502ded,0,t_0db93ef0-06e8-460a-852a-9cf2752ae224,1,t_675dd421-0c06-4914-8beb-329eff278481,0,t_ac6deb7f-8a35-4db3-840f-4f06a7a1aab0,1
4,53_0546a593-43e6-4139-af76-bfd03c6da8fa,t_66c7eb2a-fa7e-4c56-ad55-3ee477cba71e,0,t_5704cdc3-c455-440c-868c-6a880b4c4476,0,t_6c743c12-884b-4164-b80b-895c17e700f9,1,t_3d0626fc-bf86-45e0-a61d-c49250b52771,1,t_0ba77058-5e81-4696-9bd4-c21cff3f3e82,...,t_b3c6727e-3253-4c10-a6d2-a3227f6bdc74,1,t_9d410287-c57c-4513-af4b-c0f0ea046889,1,t_8223208a-2bbf-4433-8a8c-9b8b7be92ade,1,t_01b5c742-7e2f-402f-9f54-1b4f6c04af95,1,t_821c2943-624e-4ba3-ace1-360a4c4b6e35,1


In [5]:
# Loading the autoencode model created to encode state space.

session_dict = session_df.set_index('session_id').T.to_dict('list')
state_encoder = pickle.load(open('pickle/session_state_encoder.sav', 'rb'))

Follwoing function creates a state given list of songs and action performed on that song. To generate ths state stirng, it appends the track_id and action performed with "-" and encoded state is generated using the trained autoencoder.

In [11]:
def construct_state(track_and_reward_list):
    state_matrix = []
    for i in range(5):
        latent_track = latent_track_features_dict[track_and_reward_list[2*i]].copy()
        latent_track.append(track_and_reward_list[2*i + 1])
        state_matrix.append(latent_track)
        
    np_state_matrix = np.array(state_matrix)
    np_state_matrix = np_state_matrix[newaxis, :, :]
    encoded_state = state_encoder.predict(np_state_matrix)
    strings = [str(data) for data in track_and_reward_list]
    state_string = "-".join(strings)
    
    return state_string, encoded_state[0]

Following function evaluates the distance between 2 points in an n-dimensional space. This function is used to find the closest point for state and action approximation.

In [12]:
def get_distance(a, b):
    distance = 0.0
    for i in range(len(a)):
        distance += (a[i] - b[i])**2
    return math.sqrt(distance)

### Policy - 
Policy will lookup at the Q table for a state and will return the action with max Q value. If the Q table is empty, it will return a random action.

If Q table is not empty but state is not present in Q table, the policy function will find the state closest to the current state and return the action based on that state. In this way, we are able to approximate a state using nearest neighbour.

In [13]:
def policy(Q, current_state, action_list, state_vector_dict):
    action_values = Q.get(current_state, dict())
    if len(action_values) != 0:
        return max(action_values, key = action_values.get)
    if len(Q) != 0:
        closest_state = None
        minimum_distance = float('-inf')
        current_state_vector = state_vector_dict[current_state]
        for state in Q:
            dist = get_distance(state_vector_dict[state], current_state_vector)
            if dist < minimum_distance:
                minimum_distance = dist
                closest_state = state
        closest_state_action_values = Q.get(closest_state, dict())
        if len(closest_state_action_values) != 0:
            return max(closest_state_action_values, key = closest_state_action_values.get)
    return random.choice(action_list)

### SARSA Algorithm for Reinforcement Learning

#### Note: This approach is not optimized and therefore will take months to train. Therefore, we explored in other part of this pro some state approximation method

In [15]:
Q = dict()
state_vector_dict = dict()
action_list = list(latent_track_features_dict.keys())
alpha = 0.5
gamma = 1.0
episode_count = 0
session_action_error_list = []
for session_id in session_dict:
    track_and_reward_list = session_dict[session_id]
    error_list = []
    for i in range(14):
        current_state, encoded_current_state = construct_state(track_and_reward_list[2*i:2*i + 10])
        state_vector_dict[current_state] = encoded_current_state
        current_action_track = track_and_reward_list[2*i + 10]
        reward = track_and_reward_list[2*i + 11]
        
        recommended_action_track = policy(Q, current_state, action_list, state_vector_dict)
        error_list.append(get_distance(latent_track_features_dict[current_action_track], latent_track_features_dict[recommended_action_track]))
        
        next_state, encoded_next_state = construct_state(track_and_reward_list[2*i + 2 : 2*i + 10 + 2])
        state_vector_dict[next_state] = encoded_next_state
        next_action_track = track_and_reward_list[2*i + 10 + 2]
        
        current_state_action_values = Q.get(current_state, dict())
        next_state_action_values = Q.get(next_state, dict())
        current_state_action_values[current_action_track] = current_state_action_values.get(current_action_track, 0) + alpha*(reward + gamma*next_state_action_values.get(next_action_track, 0) - current_state_action_values.get(current_action_track, 0))
        
        Q[current_state] = current_state_action_values
    
    print("Episode ", episode_count, " mean error: ", mean(error_list))
    print("Error: ", error_list)
    session_action_error_list.append(error_list)
    pickle.dump(session_action_error_list, open('pickle/session_action_error_list.sav', 'wb'))
    episode_count += 1

Episode  0  mean error:  0.5212392895415363
Error:  [0.48028235808506003, 0.583435926396116, 0.5722647807791837, 0.8766212828149719, 0.47907733295512533, 0.5237059474803446, 0.6444543088154484, 0.3732658309037527, 0.2630946767041131, 0.3727771299327852, 0.4809469433778967, 0.3487817034722391, 0.6872090737222694, 0.611432758142203]
Episode  1  mean error:  0.5601861784759001
Error:  [0.5422243358481192, 0.6628794008469873, 0.47846918793075566, 0.4968202806238408, 0.6097458459129432, 0.4775413305177759, 0.5249881956340161, 0.5628794461697202, 0.5339246205661726, 0.6958584708299917, 0.7787400078988783, 0.5379521144479157, 0.48936847349523477, 0.4512147879402504]
Episode  2  mean error:  0.6873131017186961
Error:  [0.5044857306257687, 1.2276736141228137, 0.7542766257238405, 0.6084372479968392, 0.6778801056983869, 0.3693892810572337, 0.6577031905338843, 0.594925121800112, 0.36066517582676677, 0.29035962141164906, 0.8776752103181346, 0.8802810540161231, 1.1296769973246479, 0.6889544476055448

Episode  25  mean error:  0.4377827636368423
Error:  [0.320420097252206, 0.3919908443771656, 0.6061396613114844, 0.3161431514594404, 0.44359971260847436, 0.612644412820193, 0.29244570064529773, 0.2468973825877094, 0.450892507493269, 0.7435336168255214, 0.7058578405723018, 0.25496309874605805, 0.5552211073104224, 0.18820955690624896]
Episode  26  mean error:  0.7335135980385395
Error:  [1.4498277389545908, 0.7115887136949731, 0.2869643815759468, 0.21821204626137886, 0.6615928818580267, 0.7805881047351622, 0.9107652184092839, 0.8480743443892041, 0.8742866067778798, 0.26262151973126574, 0.5275570134249908, 0.8575986636220455, 0.9521429325486634, 0.9273702065561412]
Episode  27  mean error:  0.49565131829190584
Error:  [0.9283341229262116, 0.28123481567737074, 0.3919745500868435, 0.456421425406182, 0.20855732790328793, 0.3186196265791637, 0.2470799800295805, 0.5538949987366187, 0.32441741294763726, 0.26277447266474147, 0.7161483390040846, 0.968515397381991, 0.5505053497227546, 0.7306406370

Episode  50  mean error:  0.6649873565183614
Error:  [0.5248794595309152, 0.6443211141121691, 1.0214157169924367, 0.5963100820570645, 0.6505195184676459, 0.6614477985208653, 0.3398803049325673, 0.6635478404088856, 0.7888221808008292, 0.8063456126730898, 0.6547427923752488, 0.7549502325744728, 0.6466329827518599, 0.5560073550590099]
Episode  51  mean error:  0.6236678145592562
Error:  [0.7189016915875341, 0.6642309460983374, 0.6528968069387636, 0.40280036332439245, 0.7319886753968682, 0.7652253541203766, 0.7237394178257992, 0.21452480065777163, 0.6412009591494727, 0.9701947705999242, 0.5290869151700954, 0.5198214640036964, 0.6453198750300919, 0.5514173639264636]
Episode  52  mean error:  0.545913417463964
Error:  [0.40356541591157946, 0.3727278686603635, 0.4139288715477412, 0.46722094717785084, 0.7743813365203507, 0.7347259366711093, 0.7823949362251339, 0.6088512331583543, 0.5386423624708324, 0.3119404479917864, 0.6883012840721033, 0.7175881951782236, 0.4516293787400926, 0.3768896301699

Episode  75  mean error:  0.5755488257699032
Error:  [0.5506841750021518, 0.6351705520670271, 0.7714977949162609, 0.7613577761841515, 0.7028832584162377, 0.49348702525500016, 0.32536667273031694, 0.2559410232860098, 0.6042962657018242, 0.6830375417077298, 0.5992973008580563, 0.6221777838937244, 0.6532161561943688, 0.39927023456578503]
Episode  76  mean error:  0.5354151318960266
Error:  [1.0080709599888857, 0.6004997976342628, 0.4018184819262767, 0.7011760957603455, 0.36666815572038924, 1.0510795588368183, 0.6840535099898956, 0.2230080733793021, 0.45906614799704654, 0.3729770478959686, 0.47968909991353875, 0.16207995017490245, 0.5654131072402871, 0.42021186008645317]
Episode  77  mean error:  0.7018301250491625
Error:  [0.41326058217431966, 0.9320406096908945, 0.46773752644062805, 1.1085533728577504, 0.5057698619105746, 1.0644798504449073, 0.44568376953736305, 0.25397896695087224, 0.7080797053704387, 0.9473673794074369, 0.9480763199291212, 0.6611347557577011, 0.4775486624182158, 0.8919

Episode  100  mean error:  0.42547625477455897
Error:  [0.304750054485686, 0.5699032520092914, 0.40857340367969186, 0.30084453582462894, 0.30945194830087747, 0.1349825118721216, 0.21498512906939363, 0.19949100628999317, 0.3346313971840071, 0.2600523354395106, 0.5724494821323581, 0.4545537285971439, 0.849688737913507, 1.0423100440456146]
Episode  101  mean error:  0.7901933058316959
Error:  [0.9268164740167248, 0.9660523066702629, 1.0106799437688878, 1.2037051121451816, 0.9401295979235748, 1.1182963064195195, 0.9779926157091994, 0.3545387827809404, 0.43662636480530226, 0.47157838273251473, 0.6802768075178471, 0.5054625754939133, 0.4334044542463762, 1.0371465574134984]
Episode  102  mean error:  0.5331844730374722
Error:  [0.5623878892139551, 0.3424524201874999, 0.6245067123162522, 0.9579816441255387, 0.7226995977042268, 0.3584686790805565, 0.3541731847149832, 0.6061618427065976, 0.595146888505428, 0.5248152382576011, 0.2903832100266254, 0.3919124769159128, 0.4949493131348988, 0.63854352

...

Episode  3108  mean error:  0.6490689365734604
Error:  [0.8154634612165671, 0.9043789987683937, 0.4377183796723923, 0.7263456140945236, 0.15504747016897424, 0.5330021923967188, 0.8719785665577658, 0.7176064256260619, 0.8204451391688417, 0.6088150813727151, 0.6987569782385363, 0.7472854394990115, 0.3653663909602124, 0.684754974287731]
Episode  3109  mean error:  0.5927624375627271
Error:  [0.540200965740635, 1.01391668172482, 1.0526912636759465, 0.21987400926051365, 0.3756934089424015, 0.29538944373717907, 0.754604391075233, 0.35964844332605783, 0.7168375922754895, 0.47085334185649935, 0.6327659888522575, 0.8378153473786009, 0.693662836861229, 0.3347204111713166]
Episode  3110  mean error:  0.5257865752917067
Error:  [0.6277348798028495, 0.638727719591312, 0.2687625705863906, 0.4154437153548046, 0.5476704574180974, 0.4193396105944869, 0.3986392778768479, 0.98408699101343, 0.39482043275997114, 0.4673277015633057, 0.703030416659929, 0.6493708330427117, 0.7552727730973889, 0.09078467472236

Episode  3133  mean error:  0.6305413234597123
Error:  [0.9842312299203869, 0.9302427891398113, 0.6478062806748236, 0.9290612372322824, 0.2171018171992901, 0.8116931615163958, 0.10063073375579212, 1.3183095429733562, 0.8133250852493933, 0.4267803643845228, 0.5775080932457238, 0.3881193521896867, 0.2299495435170239, 0.4528192974374831]
Episode  3134  mean error:  0.6263668882737736
Error:  [0.892119978407456, 0.7270193128936095, 0.7560878510114545, 0.7533430509749879, 0.4490338629554556, 0.48854101061868366, 0.7528902077094696, 0.6897123817671066, 0.22485153396359414, 0.6061777344027619, 0.5741677474242352, 0.796655423406423, 0.6124553928976763, 0.44608094739991616]
Episode  3135  mean error:  0.556343189660447
Error:  [0.2950407186860251, 0.931543985546152, 0.38375394978225985, 0.8076405157527962, 0.5829121827669548, 0.7827619010946151, 0.2796171505656177, 0.9101258009779192, 0.720922648470396, 0.419392625519117, 0.32679781452259665, 0.4927266108517088, 0.40772583054300265, 0.447842920

Episode  3158  mean error:  0.601311609692892
Error:  [0.12444138744847459, 1.0385582160433742, 0.23858972386334554, 0.1805713676892731, 0.8670248364318214, 0.09221363611329328, 0.6752118186317666, 1.1238167309555915, 0.6802667118219788, 0.7853827973476502, 0.5550466732950757, 0.4994341550482074, 1.2600000980777082, 0.2978043829329274]
Episode  3159  mean error:  0.6655497104854199
Error:  [1.213182131159451, 0.8665143588107526, 0.4464259345803206, 0.38169414775853683, 0.18176765215952184, 0.5302002260233665, 0.8511456135623456, 0.6793509373626573, 0.9017397313509021, 0.7321221784420177, 0.48529666162974, 0.8163965219299344, 0.7594523587457263, 0.4724074932806051]
Episode  3160  mean error:  0.5646433870729577
Error:  [0.6656184461793103, 0.1664485299789011, 0.6781967601846289, 0.7319595460753295, 0.668479642798889, 0.4516912394500083, 0.7779571961094122, 0.7834839549005683, 0.5299257015886181, 0.3251275474244991, 0.7241920437333187, 0.5157467384446661, 0.4641798653784313, 0.4220002067

Episode  3183  mean error:  0.5946691511158503
Error:  [0.6512743081586454, 0.5304984960233905, 0.7456036391969575, 0.630841749872369, 0.8447857017864953, 0.6644481136318526, 0.5931410966541443, 0.32041846419203884, 0.49065367113458425, 0.16920119522673957, 0.8081690512291302, 0.7936717327984729, 0.5320873090044738, 0.5505735867126106]
Episode  3184  mean error:  0.6059999781686402
Error:  [0.6687383189903571, 0.5544136679720183, 0.6378596748351838, 0.544292914273897, 0.7269770274794645, 0.5852437040010763, 0.5216231989173944, 0.4762805801718005, 0.6021346109372093, 0.3139782731167291, 0.416733464439151, 0.6878704944244207, 0.5391575796771505, 1.2086961851251103]
Episode  3185  mean error:  0.5332266333029179
Error:  [0.6910575602444623, 0.30360910179637307, 0.7780062424920771, 0.3128047107039282, 0.2984223316842045, 0.3068765691791856, 0.7057255305508958, 0.8475081536608823, 0.765899169976516, 0.628345603996869, 0.5084654203816505, 0.3962814684655484, 0.1979003142876197, 0.72427068882

KeyboardInterrupt: 